## Installing required packages

In [2]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

Defaulting to user installation because normal site-packages is not writeable


# Fine-tuning a pre-trained model on a translation task

In this notebook, I will fine-tune one of the hugging-face Transformers model for a English to Vietnamese translation task. I will use the mt_eng_vietnamese dataset and NLP/opus-mt-en-vi model.

Reference: 
- Model: https://huggingface.co/Helsinki-NLP/opus-mt-en-ro
- Data: https://huggingface.co/datasets/mt_eng_vietnamese

## Loading the dataset

In [45]:
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("mt_eng_vietnamese", "iwslt2015-en-vi")
metric = load_metric("sacrebleu")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


The dataset object itself is [datasetdict](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [46]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 133318
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1269
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1269
    })
})

In [47]:
raw_datasets["train"][0]

{'translation': {'en': 'Rachel Pike : The science behind a climate headline',
  'vi': 'Khoa học đằng sau một tiêu đề về khí hậu'}}

To get a sense of how the data looks like, the following function will show some examples picked randomly in the dataset.

In [48]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'Nobody &apos;s ever done it before , so I &apos;m going to go do it .', 'vi': 'Chưa ai từng làm việc đó , thế nên tôi sẽ làm nó .'}"
1,"{'en': 'Now we don &apos;t know exactly what a bee sees , any more than you know what I &apos;m seeing when I call this red .', 'vi': 'Bây giờ chúng ta không biết chính xác con ong nhìn thấy gì , bạn chỉ biết cái tôi đang nhìn thấy khi tôi gọi nó là màu đỏ .'}"
2,"{'en': 'And at that moment , the dog transformed into the Buddha of love and kindness .', 'vi': 'Và vào thời điểm đó , con chó đã được cảm hoá trở thành Phật của tình yêu và lòng tốt .'}"
3,"{'en': 'Well , when I started working there , about one percent of children were fully immunized .', 'vi': 'Khi tôi bắt đầu làm việc ở đó , chỉ có khoảng 1 % trẻ em được tiêm chủng đầy đủ .'}"
4,"{'en': 'How are you going to spend these seven and a half minutes of bonus life ?', 'vi': 'Quý vị sẽ sử dụng bảy phút rưỡi phần thưởng cuộc sống này như thế nào ?'}"


# Preprocessing the data

Before I can feed those texts to our model, I need to preprocess them. This is done by a Transformers Tokenizer which will tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

In [44]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-vi"

In [49]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


This is for testing how the tokenizer would work.

In [16]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[125, 778, 3, 63, 141, 9191, 23, 0], [187, 32, 716, 9191, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the as_target_tokenizer context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets.

In [17]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[10334, 1204, 3, 15, 8915, 27, 452, 59, 29579, 581, 23, 0], [235, 1705, 11, 32, 8, 1205, 5305, 59, 29579, 581, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


The function below will preprocess our samples. I just feed them to the tokenizer with the argument truncation=True. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [50]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "vi"
def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [51]:
preprocess_function(raw_datasets['train'][:2])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[2697, 19679, 302, 53, 100, 3753, 1769, 15, 8122, 21018, 0], [338, 1076, 1642, 302, 4, 33730, 24597, 2697, 19679, 5349, 15, 16436, 13, 7, 8568, 7178, 3754, 1769, 7, 10789, 22644, 56, 8122, 1333, 302, 4, 68, 184, 2158, 777, 136, 13, 3340, 174, 12707, 777, 1539, 15, 15275, 4528, 352, 7, 31076, 22, 10027, 13, 3451, 56, 15, 2556, 18930, 302, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[9811, 266, 2378, 264, 27, 800, 381, 84, 862, 1802, 0], [553, 1076, 1158, 109, 1994, 324, 4905, 266, 862, 3529, 2697, 19679, 446, 3136, 842, 5683, 84, 46, 3007, 676, 1615, 266, 16186, 83, 232, 2378, 264, 46, 800, 381, 4648, 2137, 84, 860, 534, 862, 1802, 109, 192, 50, 1980, 1761, 509, 21, 110, 5, 33, 460, 1741, 26, 30, 6958, 4434, 34, 915, 1220, 44, 5, 33, 27, 2163, 911, 4031, 1172, 254, 2478, 1544, 77, 

To apply this function on all the pairs of sentences in our dataset, I just need use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

In [52]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

## Loading the pre-trained model

For this step, I need to download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the AutoModelForSeq2SeqLM class. Like with the tokenizer, the from_pretrained method will download and cache the model for us.

In [53]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

## Fine-tuning process

For text task, I use sacrebleu or bleu which is an algorithm for evaluating the quality of text which has been machine-translated from one natural language to another.

In [60]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

Checking the output of the compute function using sacrebleu metric:

In [14]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

To instantiate a Seq2SeqTrainer, I will need to define three more things. The most important is the [Seq2SeqTrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional.

In [54]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True    
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Here I set the evaluation to be done at the end of each epoch, tweak the learning rate, use the batch_size defined at the top of the cell and customize the weight decay. Since the Seq2SeqTrainer will save the model regularly and our dataset is quite large, I tell it to make three saves maximum. Lastly, I use the predict_with_generate option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

Model will save under **{model_name}-finetuned-{source_lang}-to-{target_lang}** directory

Then, I need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels.

In [55]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our Seq2SeqTrainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [56]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def my_compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Now I just need to pass all of necessary parameters along with our datasets to the Seq2SeqTrainer.

In [57]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=my_compute_metrics
)

I can now finetune our model by just calling the train method:

In [58]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.481200,1.272542,37.397600,28.455500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[53684]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[53684]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=8333, training_loss=1.525655201012728, metrics={'train_runtime': 5828.434, 'train_samples_per_second': 22.874, 'train_steps_per_second': 1.43, 'total_flos': 2417362848055296.0, 'train_loss': 1.525655201012728, 'epoch': 1.0})

In [61]:
import os
for dirname, _, filenames in os.walk('opus-mt-en-vi-finetuned-en-to-vi'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

opus-mt-en-vi-finetuned-en-to-vi/checkpoint-7500/tokenizer_config.json
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-7500/optimizer.pt
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-7500/generation_config.json
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-7500/vocab.json
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-7500/trainer_state.json
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-7500/target.spm
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-7500/config.json
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-7500/source.spm
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-7500/scheduler.pt
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-7500/training_args.bin
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-7500/rng_state.pth
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-7500/model.safetensors
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-7500/special_tokens_map.json
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-8000/tokenizer_config.json
opus-mt-en-vi-finetuned-en-to-vi/checkpoint-8000/optimizer.pt
opus-mt-en-vi-finetun

Since it takes me more than 2 hours just for running one epoch, I only train the model for one time only. Now my fine-tuned model already saved under opus-mt-en-vi-finetuned-en-to-vi/checkpoint-8000. I'm now loading the model and translate some text from English to Vietnamese.

In [68]:
from transformers import MarianMTModel, MarianTokenizer
src_text = ["My name is Jessie. I'm a senior at Knox College."]
model_name = 'opus-mt-en-vi-finetuned-en-to-vi/checkpoint-8000'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

['Tên tôi là Jessie. Tôi là học sinh năm cuối ở trường Knox.']

For me, these are pretty good translation and pretty much similar to what google translate would give.